# Library

In [1]:
import os
from typing import Dict, List, Optional, Any
import datetime
import time
import sys
sys.path.append('../')

from python_backend.config import logger, POLICY_FOLDER, GCP_PROJECT_ID, GCP_LOCATION, DOCUMENTS_BUCKET
from python_backend.storage.bigquery import get_fa_from_bigquery
from python_backend.storage.gcs import ensure_bucket_exists
from python_backend.ai.models import llm, embed_model
from python_backend.document.processor import process_document, docling_reader
from python_backend.storage.bigquery import create_bigquery_table, get_bigquery_client, ensure_processed_docs_table_exists, is_document_already_processed

import json
import re

/Users/beckyxu/Documents/GitHub/UNOPS---Remote-Sensing/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:httpx:HTTP Request: GET https://us-central1-aiplatform.googleapis.com/v1beta1/publishers/google/models/gemini-2.0-flash "HTTP/1.1 200 OK"


# Functions to run pipeline

In [2]:
# load the policy documents 
def create_policy_docs(
    policy_links: List[str] = None, 
    ) -> Dict:
    """
    Initialize indices for policy documents using the existing ChromaDB collection.
    
    Args:
        policy_links: list of policy document links (not used when loading from existing ChromaDB).

    Returns:
        List of extracted policy documents for analysis
    """
    # 1. create combinedtext for each document
    # file 1 SDG
    file_path = policy_links[0]
    docs = docling_reader.load_data(file_path)
    text_doc1_sdg = ' '.join(doc.text.strip() for doc in docs)
    
    # file 2 SDG
    file_path = policy_links[1]
    docs = docling_reader.load_data(file_path)
    text_doc2_sdg = ' '.join(doc.text.strip() for doc in docs)
    
    # file 3 RS
    file_path = policy_links[2]
    docs = docling_reader.load_data(file_path)
    text_doc1_rs = ' '.join(doc.text.strip() for doc in docs)
    
    # file 4 RS
    file_path = policy_links[3]
    docs = docling_reader.load_data(file_path)
    text_doc2_rs = ' '.join(doc.text.strip() for doc in docs)
    
    # file 5 RS ["this should be the list of IMAT Tools"]
    file_path = policy_links[4]
    docs = docling_reader.load_data(file_path)
    text_doc3_rs = ' '.join(doc.text.strip() for doc in docs)
    
    return [text_doc1_sdg, text_doc2_sdg, text_doc1_rs, text_doc2_rs, text_doc3_rs] 

In [109]:
# generate analysis results 
def answer_question_from_document_link(document_link: str, policy_doc_list) -> Dict[str, Any]:
    """
    Answer a question based on a document link and policy indices.
    First extracts information from policy documents, then uses that to analyze the project document.
    
    Args:
        document_link: The link to the project document to analyze.
        policy_doc_list: list of extracted document text from
                
    Returns:
        Dict containing the structured analysis and relevant context.
    """
    
    # Step 1: load policy context docs

    # Step 2: Download and analyze the project document 
    logger.info(f"Processing document link: {document_link}")
    try:
        processed_doc = process_document(document_link) 
        project_doc_text = processed_doc['text_doc_fa'] 
        project_doc_id = processed_doc['file_id'] 
    except Exception as e:
        logger.error(f"Error initializing project fa doc: {str(e)}")
        return None
    
    # Step 3: Extract data from the project fa and the policy docs 
    try:
        system_prompt = """ Use ReAct:
        1. **Reason**: Identify relevant project elements from project document.
        2. **Act**: Match to tool or indicators.
        3. **Reason**: Validate matches.
        4. **Act**: Format the response as JSON
        If information is missing or uncertain, include null values.
        """
        from ai.models import llm
        
            
        # 1. Create a sdg and sdg indicators json
        try:
            # Create prompt for the LLM
            prompt = f"""{system_prompt}
            You are analyzing a project document financial document for Sustainable Development Goals (SDGs) and measurable SDG indicators.
            You are given two sets of documents: 1. project document, 2. sdg indicators documents.
            The project document contains two main sections in the report: <finance> and <project description>.
            Focusing on <project description> and the sdg indicators documents, analyze:
            - What SDG goals does this project contribute to?
            - What specific SDG indicators are measurable in this project?"
            - Output a nested json with keys: "sdg_goals" and "sdg_indicators". Each key can have a list of objects. Follow the format below.

            *Here is the project document*:
            {project_doc_text}

            *Here are the sdg indicators documents*:
            {policy_doc_list[0]}
            {policy_doc_list[1]}

            *Strictly follow this json response format*
            {{
                "sdg_goals": [
                    {{
                        "sdg_goal": "string",  # e.g., "6"
                        "name": "string",     # e.g., "Clean Water and Sanitation"
                        "relevance": "string" # e.g., "Provides safe water"
                    }}
                ],
                "sdg_indicators": [
                    {{
                        "sdg_indicator": "string",
                        "description" : "string",
                        "measurability" : "string"
                    }}
                ]
            }}

            Only output the json string.
            """
            # Get response from LLM
            response = llm.complete(prompt)
            answer_sdg = response.text.strip()
            
            # Store in summary
            # print(answer_sdg)
            logger.info(f"Generated SDG for doc")  
        except Exception as e:
            logger.error(f"Error generating SDG doc: {str(e)}")
            answer_sdg = None
            
        # 2. Create a imat and remote sensing capacity json
        try:
            # Create prompt for the LLM
            prompt = f"""{system_prompt}
            You are an expert remote sensing analyst tasked with identifying relevant remote sensing tools for a given project.  You will be provided with two sets of information:

            1.  **Remote Sensing Tool Documents:** {policy_doc_list[4]}

            2.  **Project Description:** {project_doc_text}

            **Your Task:**

            Based on the provided Remote Sensing Tool Documents and Project Description, identify a list of remote sensing tools that are *relevant* to the project.  Relevance is determined by the project's objectives, data requirements, analysis methods, and anticipated outcomes as described in the Project Description.

            Here are some more context information on how remote sensing tools have been useful and relevant to other projects.
            {policy_doc_list[2]}
            {policy_doc_list[3]}
            
            For *each* remote sensing tool you identify as relevant, provide the following information in a structured format:

            *   **Technology:**  The exact name of the remote sensing tool (tool name) as it appears in the Remote Sensing Tool Documents. Here is a list of all tool names1. Satellite Imagery & Geospatial Analysis; 2. AI-Powered Remote Sensing; 3. UAV & Aerial Data Integration; 4. Remote Monitoring & Early Warning; 5. Interactive Dashboards & Decision Support
            *   **Relevance Justification:** A concise explanation of *why* this tool is relevant to the project.  This explanation *must* be directly supported by specific references to the Project Description.
            *   **Project Description Context:**  List the *exact* section/paragraph/page number and sentence(s) from the Project Description that support your relevance justification.  Be as specific as possible (e.g., "Section 2.3, Paragraph 4, Sentence 1-2").  Quote the relevant sentence(s) directly. If the relevancy stems from multiple location in the project description, list all of them.

            **Output Format:**
            Present your findings in a nested json format as follows. Strictly follow the format:
                {{
                    "remote_sensing_tools":[
                        {{
                            "technology": "string",  
                            "relevance_justification": "string", 
                            "project_description_context": "string"     
                        }},
                        ...(Continue for all relevant tools)
                    ]
                }}
                
            **Important Considerations:**

            *   **Accuracy is paramount.**  Ensure that your relevance justifications are directly and demonstrably supported by the Project Description. Do not infer or assume relevance; it must be explicitly stated or strongly implied.
            *   **Specificity is key.**  Provide the *most specific* section/paragraph/page and sentence references possible. Vague references are not acceptable.
            *   **Completeness is important.** Include all relevant tools based on the provided documentation.
            *   If a tool is mentioned in the Project Description but not in the Remote Sensing Tools Documents, it is not a valid tool to be included in the output.
            *   If *no* remote sensing tools are deemed relevant based on the Project Description, state this explicitly with a brief explanation of why.  Do not invent or fabricate relevance.
            *   If a tool is relevant, but you can't find the specific section/paragraph/page/sentence, mark the tool as irrelevant and give the reason.

            Now, analyze the provided Remote Sensing Tool Documents and Project Description and generate your list of relevant tools.

            """
            
            # Get response from LLM
            response = llm.complete(prompt)
            answer_rs = response.text.strip()
            
            # Store in summary
            # print(answer_rs)
            logger.info(f"Generated remote sensing summary for doc")
        except Exception as e:
            logger.error(f"Error remote sensing summary doc: {str(e)}")
            answer_rs = None
            
        answer_sdg_dict = safe_json_parse(answer_sdg) if answer_sdg else {}
        answer_rs_dict = safe_json_parse(answer_rs) if answer_rs else {}
        
        sdg_context = json.dumps(answer_sdg_dict)
        rs_context = json.dumps(answer_rs_dict)
        
        # 3. Create a summary text, with measurable indicators from above
        try:
            # Create prompt for the LLM
            prompt = f"""{system_prompt}
                Your task is to act as an analyst reviewing a project document to assess its financial aspects, project description, potential impact on Sustainable Development Goals (SDGs), and potential application of remote sensing technologies.
                You will be provided with:
                The project document text, structured with <finance> and <project description> sections: {project_doc_text}
                A list of SDGs already identified as relevant to the project: {sdg_context}
                A list of remote sensing tools already identified as potentially relevant to the project: {rs_context}
                Your analysis will be structured in two parts:
                Part A: Information Extraction
                Based only on the provided project_doc_text, extract the following information. Focus primarily on the <project description> section, but include relevant details from <finance> where specified.
                objectives: What are the main goals and aims of the project? (Focus: <project description>)
                problems_addressed: What specific societal, economic, or environmental issues does the project intend to solve or mitigate? (Focus: <project description>)
                beneficiaries_and_impacted_groups: Who are the primary recipients of the project's benefits, and which other groups or entities will be significantly affected? (Focus: <project description>)
                anticipated_outcomes_short_and_long_term: Describe the expected results and changes the project aims to achieve in the near future and over a longer period. (Focus: <project description> and <finance>)
                quantifiable_outcomes: Identify specific, measurable results stated in the document (e.g., "5 health centers built", "training for 200 farmers", "reduction of x tons of CO2"). List all such outcomes found. (Focus: <project description> and <finance> if relevant)
                If information for a specific point is not explicitly stated in the document text, indicate this by stating "Not specified in document".
                Part B: Synthesis and Structured Output
                Synthesize the information extracted in Part A, along with the provided relevant SDGs and relevant remote sensing tools, into a concise summary and structured JSON output.
                Create a brief paragraph summary of the project. This summary should incorporate the project's main purpose, key anticipated impacts, mention the relevance to the provided SDGs, and note the potential application of the provided remote sensing tools.
                **Output Format:**
                Structure your final output strictly as a JSON object following the format below.        
                {{
                    "project_summary": "string of brief project SDG impact summary in paragraph",
                    "objectives": "string",
                    "problems_addressed": "string",
                    "beneficiaries_and_impacted_groups": "string",
                    "anticipated_outcomes_short_and_long_term": "string"
                    "quantifiable_outcome_list": ["outcome_item": "string"],
                }}
                If information is not explicitly stated in the document text, indicate this by stating "Not specified in document".
            """
            
            # Get response from LLM
            response = llm.complete(prompt)
            answer_summary = response.text.strip()
            
            # Store in summary
            # print(answer_summary)
            logger.info(f"Generated summary for doc")
        except Exception as e:
            logger.info(f"Fail to generate summary {str(e)}")
            answer_summary = None
            
        answer_summary_dict = safe_json_parse(answer_summary) if answer_summary else {}
        file_id_dict = {"file_id": project_doc_id}
        combined_result = {**file_id_dict, **answer_summary_dict, **answer_sdg_dict, **answer_rs_dict}
        
        result = json.dumps(combined_result)
        return result
    
    except Exception as e:
        logger.error(f"Error processing document: {str(e)}")
        return {"answer": f"Error processing document: {str(e)}", "source_links": [document_link]}
    
# helper function
def safe_json_parse(response_text: str) -> dict:
    """
    Extracts JSON from a code block in an LLM response and parses it.
    
    Args:
        response_text: The text returned by the LLM (possibly with ```json ... ```)

    Returns:
        Parsed Python dictionary, or empty dict if parsing fails.
    """
    try:
        # Match JSON block wrapped in triple backticks
        match = re.search(r"```(?:json)?\s*({.*?})\s*```", response_text, re.DOTALL)
        if match:
            json_str = match.group(1)
            return json.loads(json_str)
    except Exception as e:
        print(f"Failed to extract/parse JSON: {e}")
    return {}

In [4]:
# file_path = policy_links[0]
# docs = docling_reader.load_data(file_path)
# text_doc1_sdg = ' '.join(doc.text.strip() for doc in docs)
# print(text_doc1_sdg)

# TEST: Process 1 Document

In [5]:
# # NOTE ONLY RUN THIS ONCE

# policy_links = [
#     'policy_docs/Global-Indicator-Framework-after-2024-refinement-English.xlsx',
#     'policy_docs/Guidance for aligning UNOPS engagements to SDGs.docx',
#     'policy_docs/Rescuing the SDGs with Geospatial Information (1).pdf',
#     'policy_docs/UNOPS Geographic Information Systems (GIS).pptx',
#     'policy_docs/IMAT’s Remote Sensing Capacities and Tools.docx',
# ]
# policy_doc_list = create_policy_docs(policy_links)

/Users/beckyxu/Documents/GitHub/UNOPS---Remote-Sensing/venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
INFO:docling.document_converter:Going to convert document batch...
INFO:docling.document_converter:Initializing pipeline for SimplePipeline with options hash 4cc01982ae99b46a2a63fcda46c47c35
INFO:docling.pipeline.base_pipeline:Processing document Global-Indicator-Framework-after-2024-refinement-English.xlsx
INFO:docling.backend.msexcel_backend:Processing sheet: A.RES.71.313 Annex
INFO:docling.document_converter:Finished converting document Global-Indicator-Framework-after-2024-refinement-English.xlsx in 0.36 sec.
INFO:docling.document_converter:Going to convert document batch...
INFO:docling.pipeline.base_pipeline:Processing document Guidance for aligning UNOPS engagements to SDGs.docx
INFO:docling.document_converter:Finished converting document Guidance for aligning UNOPS engagements 

In [88]:
document_link = "https://drive.google.com/file/d/1BnxsDqskNB2Q4KLcZ7mAK-tZ6BGXaiEY"
# add back for future use 
if not is_document_already_processed(document_link):
    result = answer_question_from_document_link(document_link, policy_doc_list)

INFO:config:Processing document link: https://drive.google.com/file/d/1BnxsDqskNB2Q4KLcZ7mAK-tZ6BGXaiEY
INFO:python_backend.document.processor:Processing document: https://drive.google.com/file/d/1BnxsDqskNB2Q4KLcZ7mAK-tZ6BGXaiEY
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:python_backend.document.processor:Successfully found file: Draft UNOPS Agreement 8.docx
INFO:python_backend.document.processor:Download progress: 100%
INFO:python_backend.document.processor:Downloaded Draft UNOPS Agreement 8.docx to /var/folders/j2/yjk_0cz112g3l8vv2_013tmr0000gn/T/tmpw_d3vi8h.docx
INFO:docling.document_converter:Going to convert document batch...
INFO:docling.pipeline.base_pipeline:Processing document tmpw_d3vi8h.docx
INFO:docling.document_converter:Finished converting document tmpw_d3vi8h.docx in 0.31 sec.
INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:httpx:HTTP Request: POST https://us-central1-aiplatform.googleapis.com/v1

----

# Create BQ table to store analysis results if not exists

In [76]:
# # # should exist already
# create_bigquery_table()

Created table unops-eil-sdg-measurement-prod.Legal_Agreements.analysis_results


# Upload to BQ

In [ ]:
from ast import main
import os
from google.cloud import bigquery
from config import logger, GCP_PROJECT_ID, BQ_FA_DATASET, BQ_FA_TABLE 
from storage.bigquery import schema, upload_to_bigquery, get_bigquery_client, is_document_already_processed

In [89]:
result_json = json.loads(result)
result_json

{'file_id': 'https://drive.google.com/file/d/1BnxsDqskNB2Q4KLcZ7mAK-tZ6BGXaiEY',
 'project_summary': 'The project aims to strengthen the capacity of maritime law enforcement agencies in Asia Pacific island countries to effectively control borders and combat transnational organized crime at sea. Key anticipated impacts include enhanced maritime security, improved border management, and contributions to Sustainable Development Goal 16 (Peace, Justice and Strong Institutions) and Goal 14 (Life Below Water). Remote sensing tools such as satellite imagery and geospatial analysis, geospatial data processing, and change detection analysis can be applied to assess equipment deployment areas, analyze maritime data, and monitor border security.',
 'objectives': 'The main objectives of the project are to support the Global Maritime Crime Programme by procuring specialized equipment and services for maritime law enforcement agencies in Fiji, Kiribati, Nauru, Tuvalu, Vanuatu, Marshall Islands, and 

In [90]:
upload_to_bigquery(result_json)

INFO:config:Uploaded row for https://drive.google.com/file/d/1BnxsDqskNB2Q4KLcZ7mAK-tZ6BGXaiEY to unops-eil-sdg-measurement-prod.Legal_Agreements.analysis_results


True

# batch processing

In [102]:
# ACTUALLY NO NEED IN THE FUTURE, JUST IMPORT FROM BIGQUERY.PY
from python_backend.auth.credentials import credentials_manager
from python_backend.config import logger, BQ_FA_DATASET, BQ_FA_TABLE 
print(BQ_FA_DATASET)
print(BQ_FA_TABLE)
def get_fa_from_bigquery(number_entries=10):
    """
    Retrieve document links from BigQuery.
    
    Args:
        number_entries (int): Number of entries to retrieve.
    
    Returns:
        List of document links (GCS URIs or Google Drive links)
    """
    try:
        # Use a different client for the project
        bigquery_client_fa = get_bigquery_client()
        if not bigquery_client_fa:
            logger.error("BigQuery client not initialized")
            return []
        
        # Query to retrieve document links
        query = f"""
        SELECT 
            -- Legal_Agreement,
            t0.File_URL,
            -- t1.Donor,
        FROM 
            `{GCP_PROJECT_ID}.{BQ_FA_DATASET}.{BQ_FA_TABLE}`
            CROSS JOIN
            UNNEST(Legal_Agreement_Files) AS t0
            -- CROSS JOIN
            --     UNNEST(Donors) AS t1
        WHERE
            t0.File_URL IS NOT NULL
        LIMIT {number_entries}
        """
        
        query_job = bigquery_client_fa.query(query) 
        results = query_job.result()
        
        links = [row["File_URL"] for row in results]
        logger.info(f"Retrieved {len(links)} document links from BigQuery")
        return links
    except Exception as e:
        logger.error(f"Error retrieving document links from BigQuery: {str(e)}")
        return []

NS_Project_Legal_Agreement
TBL_Project_Legal_Agreement


In [165]:
# .  GET A LIST OF DOCUMENT LISTS 
links = get_fa_from_bigquery(2500) # TODO: Make this function anti join with processed_documents table
links_copy = links.copy()

INFO:python_backend.config:Retrieved 2500 document links from BigQuery


In [171]:
# 2. anti join the link with those on processed_documents table
query = f"""
SELECT file_link
FROM `{GCP_PROJECT_ID}.{BQ_MIT_DATASET}.processed_documents`
"""
bigquery_client = get_bigquery_client()
query_job = bigquery_client.query(query)
processed_links = query_job.result()
processed_links = [row.file_link for row in processed_links]

links_copy = [link for link in links_copy if link not in processed_links]

In [176]:
len(links_copy)

1

In [179]:
import json

for document_link in links_copy:  # Iterate over a copy if you plan to remove items
    if not is_document_already_processed(document_link):
        result = answer_question_from_document_link(document_link, policy_doc_list)   
        #  if result is None, continue
        print(result)
        if result is None:
            mark_document_as_processed(document_link, "failed", "PermissionError")
            links_copy.remove(document_link)
            continue
        
        else:
            # Ensure result is a dict, or parse if it's a JSON string
            if isinstance(result, str):
                try:
                    result_json = json.loads(result)
                except Exception:
                    logger.error(f"Error with json.loads for {document_link}. Skipping.")
            elif isinstance(result, dict):
                result_json = result
            else:
                logger.error(f"Result for {document_link} is neither dict nor JSON string. Skipping.")
            
            # Upload the result to BigQuery 
            try:
                upload_to_bigquery(result_json)
                logger.info(f"Uploaded document link {document_link} to BigQuery. Taking a break for 10 sec to reset LLM")
                links_copy.remove(document_link)
                time.sleep(10)
            except Exception as e:
                logger.error(f"Failed to upload document link {document_link} to BigQuery: {str(e)}")
                continue

In [180]:
print(f"Remaining links to process: {links_copy}")

Remaining links to process: ['https://docs.google.com/document/d/1WEjrcN6a0eQnlUnCEWNHW8Flv7C1Fuy4pqSEOnTZqyI']


In [2]:
# def get_dataset_location(dataset_id):
#     """Retrieves the location of a BigQuery dataset."""

#     bigquery_client = get_bigquery_client()
#     dataset_ref = bigquery_client.dataset(dataset_id)
#     dataset = bigquery_client.get_dataset(dataset_ref)  # Make an API request.

#     print(f"Dataset {dataset_id} is located in {dataset.location}")
#     return dataset.location

# Join BQ Tables -> Final Results

In [3]:
bigquery_client = get_bigquery_client()
# Step 1: Get the list of file_ids
query_part1 = """
SELECT DISTINCT file_id
FROM `unops-eil-sdg-measurement-prod.Legal_Agreements.analysis_results`
"""

file_ids = [row.file_id for row in bigquery_client.query(query_part1).result()]
# Ensure the file_ids are properly formatted for SQL
file_ids_str = ",".join(f"'{file_id}'" for file_id in file_ids)
file_ids_str

INFO:python_backend.config:BigQuery client initialized


"'https://drive.google.com/file/d/1hCfUaLvDTUSB-k0WG5jDcENdw-Xtq0xr','https://drive.google.com/file/d/1dd-MvRejrqD-letNUaFq6wE5yob8AEeE','https://drive.google.com/file/d/1K3uqmb6X45j_KQI0_YXzf-gX3oWLosGS','https://drive.google.com/file/d/1xbcl7J7nB__siMnsdt_LYWdbi4JQJmXQ','https://drive.google.com/file/d/1jOlFDCVkml-18XmCLgAeLE301g6Q7rzD','https://drive.google.com/file/d/1ahvBSdhsQ2mLgL6OQ5YqnAaTcV_hGSyz','https://drive.google.com/file/d/1owohktvnDM1jFm3av6ILKdwUStAsEgHb','https://drive.google.com/file/d/1a5SbL067n4ZYj9O5JZmS_98gqMS_QxJT','https://drive.google.com/file/d/1zmzpqm_U-U_mdxdXnPvwyT4dlkDE4qwd','https://drive.google.com/file/d/1bJPqNIaMdDxbCJfNuF2_w6ZS9ri_2B3b','https://docs.google.com/document/d/1ffToLauAdRbMzQPKfYTadA8myy5y5i9pIBB4Q4I8Gzc','https://docs.google.com/spreadsheets/d/10ExpU3dUGedIrriqMYnNJN5FPXudt-P5ySXZJdDTVJY','https://drive.google.com/file/d/1otkgQ3iehf915LzkpTX654lOjvv3Ooy_','https://docs.google.com/document/d/1mQKCs6v5FsBRKKmxWxp-f8B4IHAU4Fm8Z_fmQjgIsFk','

In [5]:
# Retrieve the data
retrieve_query = f"""
WITH Ranked_Legal_Agreements AS (
    SELECT
        Legal_Agreement,
        t0.File_URL,
        Region,
        Hub,
        Engagement_Description,
        t1.Donor_Description,
        t2.Project_Manager_Name,
        t2.Project_Manager_Email_Address,
        t2.Deputy_Project_Manager_Name,
        t2.Deputy_Project_Manager_Email_Address,
        ROW_NUMBER() OVER (PARTITION BY t0.File_URL ORDER BY Legal_Agreement) AS row_num
    FROM
        `unops-eil-sdg-measurement-prod.NS_Project_Legal_Agreement.TBL_Project_Legal_Agreement`
    CROSS JOIN
        UNNEST(Legal_Agreement_Files) AS t0
    CROSS JOIN
        UNNEST(Donors) AS t1
    CROSS JOIN
        UNNEST(Projects) AS t2
    WHERE
        t0.File_URL IN ({file_ids_str})
)
SELECT 
    Legal_Agreement,
    File_URL,
    Region,
    Hub,
    Engagement_Description,
    Donor_Description,
    Project_Manager_Name,
    Project_Manager_Email_Address,
    Deputy_Project_Manager_Name,
    Deputy_Project_Manager_Email_Address
FROM Ranked_Legal_Agreements
WHERE row_num = 1
"""

# Execute the query and store the result in a DataFrame
df = bigquery_client.query(retrieve_query).to_dataframe()
df.head()

/Users/beckyxu/Documents/GitHub/UNOPS---Remote-Sensing/venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1933: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,Legal_Agreement,File_URL,Region,Hub,Engagement_Description,Donor_Description,Project_Manager_Name,Project_Manager_Email_Address,Deputy_Project_Manager_Name,Deputy_Project_Manager_Email_Address
0,55530,https://drive.google.com/file/d/1K37-zKb0InM4N...,ECR,VIE,NDC Partnership - Fund Management Support,"FPS Ministry of Foreign Affairs, Foreign Trade...",Samson Asfaw TILAHUN,samsona@unops.org,Ashebir Assefa Kidane,ashebirk@unops.org
1,27085,https://drive.google.com/file/d/1BWTn_Gn0iUlJZ...,AR,LKMCO,Emergency Covid 19 related procurement for the...,Maldives,Gayan Lasika MANORATHNA,gayanm@unops.org,Wanni Arachchilage Lahiru Geethanga PERERA,lahirup@unops.org
2,24748,https://drive.google.com/file/d/1zmzpqm_U-U_md...,NYSC,NYSC_SDC,Global ICCA Support Initiative (Award ID 00082...,Other UNDP MDTF,Ada SAFANOVA,adas@unops.org,Teymur MAMMADZADA,teymurm@unops.org
3,56351,https://docs.google.com/document/d/1XgbNSsAxJH...,AFR,CDOH,Services d’appui à la mise en oeuvre d’infrast...,Madagascar,Bodi YERIMA,yerimab@unops.org,Dibaud Mulunda LUBANZA,dibaudm@unops.org
4,56878,https://drive.google.com/file/d/13yRVDmiN1mH4_...,LCR,PEOC,"Licitación por encargo del servicio de diseño,...",Peru,MARIA NOEL TERRA LOPEZ,mariate@unops.org,CONSUELO VICTORIA RIVERA CHAPARRO,victoriar@unops.org


In [6]:
# Define the temporary table name
temp_table_id = "unops-eil-sdg-measurement-prod.Legal_Agreements.project_metadata_temp"

# Load the DataFrame to the temporary table
df.to_gbq(destination_table=temp_table_id, 
          project_id="unops-eil-sdg-measurement-prod", 
          if_exists="replace")

print(f"Temporary table created: {temp_table_id}")

/var/folders/j2/yjk_0cz112g3l8vv2_013tmr0000gn/T/ipykernel_3873/1147012983.py:5: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(destination_table=temp_table_id,
71 out of 71 rows loaded.00<?, ?it/s]INFO:pandas_gbq.gbq:
100%|██████████| 1/1 [00:00<00:00, 711.26it/s]

Temporary table created: unops-eil-sdg-measurement-prod.Legal_Agreements.project_metadata_temp


In [7]:
# make the final joined table
query = """
    CREATE OR REPLACE TABLE
        `unops-eil-sdg-measurement-prod`.`Legal_Agreements`.`final_results` AS
    SELECT
        *
    FROM
        `unops-eil-sdg-measurement-prod`.`Legal_Agreements`.`project_metadata_temp`
    INNER JOIN
        `unops-eil-sdg-measurement-prod`.`Legal_Agreements`.`analysis_results` AS t0
    ON
    project_metadata_temp.File_URL = t0.file_id;
"""

bigquery_client.query(query).result()